<a href="https://colab.research.google.com/github/KoeusIss/orvp/blob/main/0x01_fe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Feature Engineering
This data science task is built on Kaggle competition released by **Optiver** to predict the realized volatility (market fluctuation).
In this step we try to explore some potential options to produce necessary features in order to maximize our chances finding a decent prediction.
To acheive this step we need some financial basics to understand our objective and the possible extra information we can extract from our datasets so we need to shed the light on 4 concept as discribed in the competition.
* bid/ask spread
* Weighted averaged price
* Log returns
* Realized volatilit

### bid/ask spread

As different stocks trade on different level on the market we take the ratio of best offer price and best bid price to calculate the bid-ask spread.

The formula of bid/ask spread can be written in below form:
$$BidAskSpread = BestOffer / BestBid - 1$$

And the spread _spread_:
$$ Spread = AskPrice -  BidPrice$$

### Weighted averaged price

The order book is also one of the primary source for stock valuation. A fair book-based valuation must take two factors into account: the level and the size of orders. In this competition we used weighted averaged price, or WAP, to calculate the instantaneous stock valuation and calculate realized volatility as our target.

The formula of WAP can be written as below, which takes the top level price and volume information into account:

$$𝑊𝐴𝑃=\frac{𝐵𝑖𝑑𝑃𝑟𝑖𝑐𝑒1∗𝐴𝑠𝑘𝑆𝑖𝑧𝑒1+𝐴𝑠𝑘𝑃𝑟𝑖𝑐𝑒1∗𝐵𝑖𝑑𝑆𝑖𝑧𝑒1}{𝐵𝑖𝑑𝑆𝑖𝑧𝑒1+𝐴𝑠𝑘𝑆𝑖𝑧𝑒1}$$
 
As you can see, if two books have both bid and ask offers on the same price level respectively, the one with more offers in place will generate a lower stock valuation, as there are more intended seller in the book, and more seller implies a fact of more supply on the market resulting in a lower stock valuation.

### Log returns
Returns are widely used in finance, however log returns are preferred whenever some mathematical modelling is required. Calling  $𝑆_𝑡$  the price of the stock  $𝑆$  at time  $𝑡$ , we can define the log return between  $𝑡_1$  and  $𝑡_2$  as:
$$𝑟_{𝑡1,𝑡2}=\log(𝑆_{𝑡2}, 𝑆_{𝑡1})$$
 
Usually, we look at log returns over fixed time intervals, so with 10-minute log return we mean  $𝑟_𝑡=𝑟_{𝑡−10 𝑚𝑖𝑛,𝑡}$ .

### Realized volatility
When we trade options, a valuable input to our models is the standard deviation of the stock log returns. The standard deviation will be different for log returns computed over longer or shorter intervals, for this reason it is usually normalized to a 1-year period and the annualized standard deviation is called **volatility**.
We will compute the log returns over all consecutive book updates and we define the realized volatility,  $\sigma$ , as the squared root of the sum of squared log returns. $$\sigma=\sqrt{\Sigma_t r^2_{t-1,t}}$$

In [1]:
# # Connect with GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import seaborn as sns

# Constant
BASE_PATH = "/content/drive/MyDrive/DS/optiver-realized-volatility-prediction/"
TEST_FILE = BASE_PATH + "test.csv"
TRAIN_FILE = BASE_PATH + "train.csv"

In [4]:
# Financial utilities functions

def wap(df, index):
    """
    Compute the Waighted Avergae Price
    """
    bid_price = df[f"bid_price{index}"]
    bid_size = df[f"bid_size{index}"]
    ask_price = df[f"ask_price{index}"]
    ask_size = df[f"ask_size{index}"]
    return bid_price * ask_size + ask_price * bid_size / (bid_size + ask_size)

def log_return(series):
    """
    Compute the log_return of WAP
    """
    return np.log(series).diff()

def realized_volatility(series):
    """
    Compute the realized volatility
    """
    return np.sqrt(np.sum(series**2))

def total_volume(df):
    """
    Compute the total volume between asks and bids
    """
    ask_size_1 = df["ask_size1"]
    ask_size_2 = df["ask_size2"]
    bid_size_1 = df["bid_size1"]
    bid_size_2 = df["bid_size2"]
    return ask_size_1 + ask_size_2 + bid_size_1 + bid_size_2

def imbalance_volume(df):
    """
    Computes the imbalance volume
    """
    ask_size_1 = df["ask_size1"]
    ask_size_2 = df["ask_size2"]
    bid_size_1 = df["bid_size1"]
    bid_size_2 = df["bid_size2"]
    return np.abs(ask_size_1 + ask_size_2 - bid_size_1 - bid_size_2)

def spread_price(df, index):
    """
    Compute the spread
    """
    ask_price = df[f"ask_price{index}"]
    bid_price = df[f"bid_price{index}"]
    return ask_price - bid_price

def bid_ask_spread(df, index):
    """
    Compute the bid/ask spread
    """
    spread_price = df[f"spread_price{index}"]
    bid_price = df[f"bid_price{index}"]
    return spread_price / bid_price

In [5]:
def set_book_features(df, window_size=100):
    """
    Set book features // Windowing rows on columns
    """
    # First level aggregation
    df["wap_1"]                = wap(df, 1)
    df["wap_2"]                = wap(df, 2)
    df["log_return_1"]         = df.groupby("time_id")["wap_1"].apply(log_return)
    df["log_return_2"]         = df.groupby("time_id")["wap_2"].apply(log_return)
    df["spread_price_1"]       = spread_price(df, 1)
    df["spread_price_2"]       = spread_price(df, 2)
    df["bid_ask_spread_1"]     = spread_price(df, 1)
    df["bid_ask_spread_2"]     = spread_price(df, 2)
    df['wap_balance']          = np.abs(df['wap_1'] - df['wap_2'])
    df["total_volume"]         = total_volume(df)
    df["imbalance_volume"]     = imbalance_volume(df)
    
    # Second level aggregation
    aggregation_dict = {
        "log_return_1": [realized_volatility],
        "log_return_2": [realized_volatility],
        "wap_1": [np.mean, np.std],
        "wap_2": [np.mean, np.std],
        "bid_price1": [np.mean, np.max],
        "bid_price2": [np.mean, np.max],
        "ask_price1": [np.mean, np.min],
        "ask_price2": [np.mean, np.min],
        "bid_size1": [np.mean, np.sum],
        "bid_size2": [np.mean, np.sum],
        "ask_size1": [np.mean, np.sum],
        "ask_size2": [np.mean, np.sum],
        "spread_price_1": [np.mean, np.std],
        "spread_price_2": [np.mean, np.std],
        "bid_ask_spread_1": [np.mean, np.std],
        "bid_ask_spread_2": [np.mean, np.std],
        "wap_balance": [np.mean, np.std],
        "total_volume": [np.mean, np.std],
        "imbalance_volume": [np.mean, np.std]
    }
    features_lst = []
    for second in range(0, 600, window_size):
        df_feature = df[df['seconds_in_bucket'] >= second].groupby("time_id").agg(aggregation_dict)
        df_feature.columns = ['_'.join(col) + f"_s{600 - second}" for col in df_feature.columns]
        features_lst.append(df_feature)
    return pd.concat(features_lst, axis=1)

In [6]:
def set_trade_features(df, window_size=100):
    """
    Set trade features // Windowing rows on columns
    """
    # First level aggregation
    df["log_return"]   = df.groupby("time_id")["price"].apply(log_return)
    df['amount']       = df['price'] * df['size']
    
    # Second level aggregation
    aggregation_dict = {
        "log_return": [realized_volatility],
        "size": [np.mean, np.sum],
        "amount": [np.mean, np.sum],
        "order_count": [np.mean, np.sum],
    }
    features_lst = []
    for second in range(0, 600, window_size):
        df_feature = df[df['seconds_in_bucket'] >= second].groupby("time_id").agg(aggregation_dict)
        df_feature.columns = ['_'.join(col) + f"_s{600 - second}" for col in df_feature.columns]
        features_lst.append(df_feature)
    return pd.concat(features_lst, axis=1)

After setting the new features we tried to convert the time series datasets to supervised problem by windowing data by seconds_in_bucket so instead of 600 tic of seconds we regroup each 100 seconds and biuld a second level of aggregation.

In [7]:
def preprocess(ids_list, mode, window_size=60):
    """
    Preprocess dataset
    """
    def create_stock_df(stock_id):
        book = pd.read_parquet(f"{BASE_PATH}book_{mode}.parquet/stock_id={stock_id}")
        trade = pd.read_parquet(f"{BASE_PATH}trade_{mode}.parquet/stock_id={stock_id}")

        book_features = set_book_features(book, window_size)
        trade_features = set_trade_features(trade, window_size)
        
        features = book_features.join(trade_features, how='outer')
        features = features.reset_index()
        features["stock_id"] = stock_id
        return features
      
    df_list = Parallel(n_jobs=-1, verbose=1)(
        delayed(create_stock_df)(_id) for _id in ids_list
    )
    return pd.concat(df_list, ignore_index = True)

In [14]:
# Get the train file
train = pd.read_csv(TRAIN_FILE)
# For rapid and simple demonstration lets take only 5 stock_id
train = train[train["stock_id"].isin([0, 1, 2, 3, 4])]
train_ids = train.stock_id.unique()

We use paralllel computing distribution to fasten the load process

In [16]:
# Preprocess data + FE
train_df = preprocess(train_ids, "train")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.8min finished


In [17]:
train_df

,time_id,log_return_1_realized_volatility_s600,log_return_2_realized_volatility_s600,wap_1_mean_s600,wap_1_std_s600,wap_2_mean_s600,wap_2_std_s600,bid_price1_mean_s600,bid_price1_amax_s600,bid_price2_mean_s600,bid_price2_amax_s600,ask_price1_mean_s600,ask_price1_amin_s600,ask_price2_mean_s600,ask_price2_amin_s600,bid_size1_mean_s600,bid_size1_sum_s600,bid_size2_mean_s600,bid_size2_sum_s600,ask_size1_mean_s600,ask_size1_sum_s600,ask_size2_mean_s600,ask_size2_sum_s600,spread_price_1_mean_s600,spread_price_1_std_s600,spread_price_2_mean_s600,spread_price_2_std_s600,bid_ask_spread_1_mean_s600,bid_ask_spread_1_std_s600,bid_ask_spread_2_mean_s600,bid_ask_spread_2_std_s600,wap_balance_mean_s600,wap_balance_std_s600,total_volume_mean_s600,total_volume_std_s600,imbalance_volume_mean_s600,imbalance_volume_std_s600,log_return_1_realized_volatility_s540,log_return_2_realized_volatility_s540,wap_1_mean_s540,...,amount_mean_s360,amount_sum_s360,order_count_mean_s360,order_count_sum_s360,log_return_realized_volatility_s300,size_mean_s300,size_sum_s300,amount_mean_s300,amount_sum_s300,order_count_mean_s300,order_count_sum_s300,log_return_realized_volatility_s240,size_mean_s240,size_sum_s240,amount_mean_s240,amount_sum_s240,order_count_mean_s240,order_count_sum_s240,log_return_realized_volatility_s180,size_mean_s180,size_sum_s180,amount_mean_s180,amount_sum_s180,order_count_mean_s180,order_count_sum_s180,log_return_realized_volatility_s120,size_mean_s120,size_sum_s120,amount_mean_s120,amount_sum_s120,order_count_mean_s120,order_count_sum_s120,log_return_realized_volatility_s60,size_mean_s60,size_sum_s60,amount_mean_s60,amount_sum_s60,order_count_mean_s60,order_count_sum_s60,stock_id
0,5,26.486818,22.483248,75.326782,65.845229,90.506607,72.007354,1.003314,1.004267,1.003139,1.004215,1.004169,1.002301,1.004320,1.002353,78.264901,23636,80.880795,24426,74.579470,22523,89.771523,27111,0.000855,0.000212,0.001181,0.000214,0.000855,0.000212,0.001181,0.000214,62.881367,59.492011,323.496689,138.101214,134.894040,107.260583,25.717056,22.364330,74.784721,...,72.099457,1802.486450,2.400000,60.0,0.001308,75.571429,1587.0,75.839081,1592.620728,2.571429,54.0,0.001271,79.200000,1584.0,79.480331,1589.606689,2.600000,52.0,0.001121,65.312500,1045.0,65.523590,1048.377441,2.437500,39.0,0.000993,67.181818,739.0,67.390648,741.297119,2.454545,27.0,0.000953,88.375000,707.0,88.646118,709.168945,2.750000,22.0,0
1,11,14.325416,14.420526,71.729385,98.316379,95.385727,88.569580,1.000011,1.000627,0.999870,1.000477,1.000406,0.999975,1.000541,1.000176,149.965000,29993,95.445000,19089,71.145000,14229,94.895000,18979,0.000394,0.000157,0.000671,0.000200,0.000394,0.000157,0.000671,0.000200,109.262260,101.483893,411.450000,172.263581,142.050000,102.139758,14.135647,14.388569,71.402946,...,53.445446,1122.354370,2.047619,43.0,0.000587,56.250000,900.0,56.270370,900.325928,2.250000,36.0,0.000557,62.357143,873.0,62.380142,873.321960,2.142857,30.0,0.000501,82.800000,828.0,82.830833,828.308289,2.200000,22.0,0.000496,103.250000,826.0,103.288475,826.307800,2.500000,20.0,0.000203,69.333333,208.0,69.376968,208.130905,2.666667,8.0,0
2,16,10.119452,17.719946,131.348434,95.878744,75.175926,67.462624,0.999204,1.000120,0.999007,0.999928,0.999929,0.997678,1.000127,0.997966,96.132979,18073,114.526596,21531,131.037234,24635,74.654255,14035,0.000725,0.000164,0.001120,0.000295,0.000725,0.000164,0.001120,0.000295,95.771446,89.156242,416.351064,138.433034,141.414894,108.891243,10.119355,16.302774,138.973663,...,98.648781,1479.731689,3.000000,45.0,0.001137,99.083333,1189.0,98.906281,1186.875366,3.166667,38.0,0.001048,108.700000,1087.0,108.492630,1084.926392,3.400000,34.0,0.001048,120.555556,1085.0,120.325264,1082.927368,3.666667,33.0,0.000820,97.285714,681.0,97.070892,679.496216,2.428571,17.0,0.000325,105.500000,211.0,105.253487,210.506973,2.500000,5.0,0
3,31,9.231048,10.417271,121.114990,104.359625,131.333786,80.231070,0.998445,0.999815,0.998255,0.999769,0.999304,0.998519,0.999413,0.998566,114.458333,1373

Instead of 8 features we built a 432 feature describing a 6 window of time with statistics and financial features

In [18]:
# Merge and select
df_train = train.merge(train_df, on=["stock_id", "time_id"], how="left")

In [20]:
# let's check the missing data
df_train.isna().sum()

stock_id                                   0
time_id                                    0
target                                     0
log_return_1_realized_volatility_s600      0
log_return_2_realized_volatility_s600      0
                                        ... 
size_sum_s60                             722
amount_mean_s60                          722
amount_sum_s60                           722
order_count_mean_s60                     722
order_count_sum_s60                      722
Length: 433, dtype: int64

In [21]:
# Time windowing produce some missing value when there's no sufficient data in the last bucket seconds
# Forward filling the missing
df_train = df_train.ffill()

In [22]:
df_train

,stock_id,time_id,target,log_return_1_realized_volatility_s600,log_return_2_realized_volatility_s600,wap_1_mean_s600,wap_1_std_s600,wap_2_mean_s600,wap_2_std_s600,bid_price1_mean_s600,bid_price1_amax_s600,bid_price2_mean_s600,bid_price2_amax_s600,ask_price1_mean_s600,ask_price1_amin_s600,ask_price2_mean_s600,ask_price2_amin_s600,bid_size1_mean_s600,bid_size1_sum_s600,bid_size2_mean_s600,bid_size2_sum_s600,ask_size1_mean_s600,ask_size1_sum_s600,ask_size2_mean_s600,ask_size2_sum_s600,spread_price_1_mean_s600,spread_price_1_std_s600,spread_price_2_mean_s600,spread_price_2_std_s600,bid_ask_spread_1_mean_s600,bid_ask_spread_1_std_s600,bid_ask_spread_2_mean_s600,bid_ask_spread_2_std_s600,wap_balance_mean_s600,wap_balance_std_s600,total_volume_mean_s600,total_volume_std_s600,imbalance_volume_mean_s600,imbalance_volume_std_s600,log_return_1_realized_volatility_s540,...,size_sum_s360,amount_mean_s360,amount_sum_s360,order_count_mean_s360,order_count_sum_s360,log_return_realized_volatility_s300,size_mean_s300,size_sum_s300,amount_mean_s300,amount_sum_s300,order_count_mean_s300,order_count_sum_s300,log_return_realized_volatility_s240,size_mean_s240,size_sum_s240,amount_mean_s240,amount_sum_s240,order_count_mean_s240,order_count_sum_s240,log_return_realized_volatility_s180,size_mean_s180,size_sum_s180,amount_mean_s180,amount_sum_s180,order_count_mean_s180,order_count_sum_s180,log_return_realized_volatility_s120,size_mean_s120,size_sum_s120,amount_mean_s120,amount_sum_s120,order_count_mean_s120,order_count_sum_s120,log_return_realized_volatility_s60,size_mean_s60,size_sum_s60,amount_mean_s60,amount_sum_s60,order_count_mean_s60,order_count_sum_s60
0,0,5,0.004136,26.486818,22.483248,75.326782,65.845229,90.506607,72.007354,1.003314,1.004267,1.003139,1.004215,1.004169,1.002301,1.004320,1.002353,78.264901,23636,80.880795,24426,74.579470,22523,89.771523,27111,0.000855,0.000212,0.001181,0.000214,0.000855,0.000212,0.001181,0.000214,62.881367,59.492011,323.496689,138.101214,134.894040,107.260583,25.717056,...,1796.0,72.099457,1802.486450,2.400000,60.0,0.001308,75.571429,1587.0,75.839081,1592.620728,2.571429,54.0,0.001271,79.200000,1584.0,79.480331,1589.606689,2.600000,52.0,0.001121,65.312500,1045.0,65.523590,1048.377441,2.437500,39.0,0.000993,67.181818,739.0,67.390648,741.297119,2.454545,27.0,0.000953,88.375000,707.0,88.646118,709.168945,2.750000,22.0
1,0,11,0.001445,14.325416,14.420526,71.729385,98.316379,95.385727,88.569580,1.000011,1.000627,0.999870,1.000477,1.000406,0.999975,1.000541,1.000176,149.965000,29993,95.445000,19089,71.145000,14229,94.895000,18979,0.000394,0.000157,0.000671,0.000200,0.000394,0.000157,0.000671,0.000200,109.262260,101.483893,411.450000,172.263581,142.050000,102.139758,14.135647,...,1122.0,53.445446,1122.354370,2.047619,43.0,0.000587,56.250000,900.0,56.270370,900.325928,2.250000,36.0,0.000557,62.357143,873.0,62.380142,873.321960,2.142857,30.0,0.000501,82.800000,828.0,82.830833,828.308289,2.200000,22.0,0.000496,103.250000,826.0,103.288475,826.307800,2.500000,20.0,0.000203,69.333333,208.0,69.376968,208.130905,2.666667,8.0
2,0,16,0.002168,10.119452,17.719946,131.348434,95.878744,75.175926,67.462624,0.999204,1.000120,0.999007,0.999928,0.999929,0.997678,1.000127,0.997966,96.132979,18073,114.526596,21531,131.037234,24635,74.654255,14035,0.000725,0.000164,0.001120,0.000295,0.000725,0.000164,0.001120,0.000295,95.771446,89.156242,416.351064,138.433034,141.414894,108.891243,10.119355,...,1482.0,98.648781,1479.731689,3.000000,45.0,0.001137,99.083333,1189.0,98.906281,1186.875366,3.166667,38.0,0.001048,108.700000,1087.0,108.492630,1084.926392,3.400000,34.0,0.001048,120.555556,1085.0,120.325264,1082.927368,3.666667,33.0,0.000820,97.285714,681.0,97.070892,679.496216,2.428571,17.0,0.000325,105.500000,211.0,105.253487,210.506973,2.500000,5.0
3,0,31,0.002195,9.231048,10.417271,121.114990,104.359625,131.333786,80.231070,0.998445,0.999815,0.998255,0.999769,0.999304,0.998519,0.999413,0.998566,114.458333,13735,68.783333,8254,120.800000,14496,1